In [5]:
#import libraries

import pandas as pd
import string
import nltk
import re
import numpy as np
import emoji
import spacy
import gensim
import spacy
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian") 
nlp = spacy.load('ru_core_news_sm')
from nltk import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from spacy.tokenizer import Tokenizer

#upload the file

data = pd.read_csv(r'C:\Users\dell\Desktop\pliki\test_8069.csv')
data

,date,likes,comments,link to post,postfs
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите"
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...
...,...,...,...,...,...
8064,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро..."
8065,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит..."
8066,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ..."
8067,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно..."


In [6]:
data['postfs']

0                                  Добрый день,подскажите
1       Привет! Вопрос о пересечении границы Беларусь-...
2                     Также делаю квесты для детей для...
3       Здравствуйте! Планирую открыть счёт в банке PK...
4       Всем привет. Может у кого-то из вас была такая...
                              ...                        
8064    Подскажите пожалуйста,из Тересполя в Брест,кро...
8065    Добрый день, кто-нибудь знает, можно ли привит...
8066    Добрага дня, шаноўныя ! Можа тут нехта працуе ...
8067    Всем привет Подскажите, где оформить доверенно...
8068                                          Отдам даром
Name: postfs, Length: 8069, dtype: object

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8069 entries, 0 to 8068
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          8069 non-null   object
 1   likes         8069 non-null   int64 
 2   comments      8069 non-null   int64 
 3   link to post  8064 non-null   object
 4   postfs        8069 non-null   object
dtypes: int64(2), object(3)
memory usage: 315.3+ KB


In [9]:
# I stage - data preprocessing

def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)


In [10]:
def emoji_free(text):
    
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    
    return clean_text

In [11]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(emoji_free(text)))))
data['preproccessed'] = list(map(preproccessing, data['postfs']))

In [12]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(emoji_free(text.lower())))) for text in tqdm(data['postfs'])]

  0%|          | 0/8069 [00:00<?, ?it/s]

In [13]:
prep_text

['добрый день подскажите',
 'привет вопрос о пересечении границы беларусь польша',
 'также делаю квесты для детей для ',
 'здравствуйте планирую открыть счёт в банке pko для начисления зп может кто подскажет в каких отделениях pko в варшаве имеется русскоговорящий',
 'всем привет может у кого то из вас была такая ситуация в рб я числюсь как тунеядец и мне нужно предоставить документ подтверждающий ',
 'кто нибудь в курсе что за изменения в отношении выдачи карты поляка ',
 ' остеопатии ',
 'кто может выполнить комплекс строительных работ по ремонту нового помещения под пекарню бетонные малярно штукату',
 'енно вообще человек из жкх сказал что обычно ему присылают какие то справки из польской налоговой которая подтверждает уплату налогов на территории рп может знаете как называется такая справка и всем привет может у кого то из вас была такая ситуация в рб я числюсь как тунеядец и мне нужно предоставить документ подтверждающий ',
 'подскажите пожалуйста может ето знает русскоговорящег',

In [14]:
data['text_prep'] = prep_text

In [15]:
data

,date,likes,comments,link to post,postfs,preproccessed,text_prep
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите",Добрый день подскажите,добрый день подскажите
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...
...,...,...,...,...,...,...,...
8064,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...
8065,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...
8066,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...
8067,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...


In [16]:
text = data['text_prep'][1]
word_tokenize(text) 

['привет', 'вопрос', 'о', 'пересечении', 'границы', 'беларусь', 'польша']

In [17]:
# STOPWORDS my_list
with open ('C:/Users/dell/Desktop/pliki/stop.txt', 'r',  encoding='utf-8') as stop_file:
    russian_stopwords = [word.strip() for word in stop_file.readlines()]

In [18]:
stop_words = russian_stopwords

In [19]:
stop = set(stop_words)

In [20]:
# Custom stopwords
custom_stopwords = ['«','”','„', '»', '', '.', '-', '—', '$', '@']

In [21]:
custom = set(custom_stopwords)

In [22]:
type(custom)

set

In [23]:
type(stop)

set

In [24]:
# Customize stop words by adding to the default list
stop_words = stop.union(custom)


In [25]:
print(len(stop_words))

1391


In [26]:
nltk.download('punkt')
#This tokenizer divides a text into a list of sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
#II stage - removing stop words
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in stop_words and token != ' ']
    return " ".join(tokens)

In [28]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(data['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in stop_words and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

data['text_sw'] = sw_texts_list

  0%|          | 0/8069 [00:00<?, ?it/s]

In [29]:
data

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите",Добрый день подскажите,добрый день подскажите,подскажите
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...
...,...,...,...,...,...,...,...,...
8064,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...
8065,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...
8066,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...
8067,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...


In [30]:
sw_texts_list

['подскажите',
 'пересечении границы беларусь польша',
 'квесты детей',
 'планирую открыть счёт банке pko начисления зп подскажет каких отделениях pko варшаве русскоговорящий',
 'ситуация рб числюсь тунеядец предоставить подтверждающий',
 'курсе отношении выдачи карты поляка',
 'остеопатии',
 'выполнить комплекс строительных работ нового помещения пекарню бетонные малярно штукату',
 'енно жкх присылают какие справки польской налоговой подтверждает уплату налогов территории рп справка ситуация рб числюсь тунеядец предоставить подтверждающий',
 'подскажите ето русскоговорящег',
 'раздумьях какую полициальную школу выбрать start центре варшавы приглашает желающих бесплатное обучение учусь мужем знаю',
 'магчыма шукаем менавіта цябе',
 'посоветуйте тв пакет желательно работающий',
 'учусь стационаре прив',
 'подскажите варшаве удалить родинки',
 'подскажите студентам пересечении границы экспресс пцр тест ответ',
 'польской компании требуется декларант помощник декларанта клиентами прием об

In [31]:
#data.to_csv('stop_text.csv')

In [32]:
#III stage - lemmatization
mystem = Mystem() 
lemm_texts_list = []
for text in tqdm(data['text_sw']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in stop_words]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
data['text_lemm'] = lemm_texts_list

  0%|          | 0/8069 [00:00<?, ?it/s]

In [33]:
lemm_texts_list

['\n',
 'пересечение граница беларусь польша \n',
 'квеста \n',
 'планировать открывать счет банк pko начисление зп отделение pko варшава русскоговорящий \n',
 'ситуация рб числиться тунеядец предоставлять подтверждать \n',
 'отношение выдача карта поляк \n',
 'остеопатия \n',
 'выполнять комплекс строительный помещение пекарня бетонный малярный штукат \n',
 'енное жкх присылать справка польский налоговый подтверждать уплата налог территория рп справка ситуация рб числиться тунеядец предоставлять подтверждать \n',
 'ето русскоговорящег \n',
 'раздумье полициальный выбирать start центр варшава приглашать желающий бесплатный обучение учиться муж \n',
 'магчым шукать менавіта цяб \n',
 'посоветовать тв пакет желательно \n',
 'учиться стационар прив \n',
 'варшава удалять родинка \n',
 'студент пересечение граница экспресс пцр тест ответ \n',
 'польский требоваться декларант помощник декларант клиент прием обработка дежурство сменно офис число желательно опыт русскоязычный варшава район ми

In [34]:
data['text_sw'][58]

'польской компании требуется декларант помощник декларанта сменам официальное трудоустройство'

In [35]:
data['text_lemm'][58]

'польский требоваться декларант помощник декларант смена официальный трудоустройство \n'

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8069 entries, 0 to 8068
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           8069 non-null   object
 1   likes          8069 non-null   int64 
 2   comments       8069 non-null   int64 
 3   link to post   8064 non-null   object
 4   postfs         8069 non-null   object
 5   preproccessed  8069 non-null   object
 6   text_prep      8069 non-null   object
 7   text_sw        8069 non-null   object
 8   text_lemm      8069 non-null   object
dtypes: int64(2), object(7)
memory usage: 567.5+ KB


In [38]:
data.to_csv('C:/Users/dell/Desktop/pliki/lemm_8069.csv', encoding = 'utf-8',  index = False)

In [39]:
data = pd.read_csv(r'C:\Users\dell\Desktop\pliki\lemm_8069.csv')
data

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите",Добрый день подскажите,добрый день подскажите,подскажите,\n
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша \n
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста \n
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
...,...,...,...,...,...,...,...,...,...
8064,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
8065,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
8066,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...
8067,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...


In [40]:
data['text_lemm'] = data['text_lemm'].str.replace('\n','', regex=True)

In [41]:
#see all the rows with the NaN values
data[data['text_lemm'].isnull()]

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
36,2022-01-05,0,30,https://mbasic.facebook.com/groups/32275791783...,"2,не за спасибо))))",не за спасибо,не за спасибо,NaN,NaN
67,2022-01-07,0,60,https://mbasic.facebook.com/groups/32275791783...,"Здравствуйте,по",Здравствуйте по,здравствуйте по,NaN,NaN
68,2022-01-07,6,60,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте!!!,Здравствуйте,здравствуйте,NaN,NaN
231,2022-01-11,45,140,https://mbasic.facebook.com/groups/32275791783...,!!!,,,NaN,NaN
261,2022-01-12,24,0,https://mbasic.facebook.com/groups/32275791783...,.,,,NaN,NaN
269,2022-01-12,1,0,https://mbasic.facebook.com/groups/32275791783...,я сторона:),я сторона,я сторона,NaN,NaN
327,2022-01-13,0,20,https://mbasic.facebook.com/groups/32275791783...,Всем здравствуйте!,Всем здравствуйте,всем здравствуйте,NaN,NaN
342,2022-01-13,69,60,https://mbasic.facebook.com/groups/32275791783...,Добрый день!,Добрый день,добрый день,NaN,NaN
353,2022-01-14,0,10,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте,Здравствуйте,здравствуйте,NaN,NaN
412,2022-01-14,7,80,https://mbasic.facebook.com/groups/32275791783...,Добрый день,Добрый день,добрый день,NaN,NaN


In [42]:
# using dropna() function  to delete nan values
data.dropna()

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите",Добрый день подскажите,добрый день подскажите,подскажите,
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
...,...,...,...,...,...,...,...,...,...
8064,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
8065,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
8066,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...
8067,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...


In [43]:
# Reset index after drop
data=data.dropna().reset_index(drop=True)

In [44]:
#data.drop(data.filter(regex="Unname"),axis=1, inplace=True)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8015 entries, 0 to 8014
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           8015 non-null   object
 1   likes          8015 non-null   int64 
 2   comments       8015 non-null   int64 
 3   link to post   8015 non-null   object
 4   postfs         8015 non-null   object
 5   preproccessed  8015 non-null   object
 6   text_prep      8015 non-null   object
 7   text_sw        8015 non-null   object
 8   text_lemm      8015 non-null   object
dtypes: int64(2), object(7)
memory usage: 563.7+ KB


In [46]:
data.to_csv('C:/Users/dell/Desktop/pliki/lemm_8015.csv', encoding = 'utf-8',  index = False)

In [47]:
data = pd.read_csv(r'C:\Users\dell\Desktop\pliki\lemm_8015.csv')
data

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-02,0,130,https://mbasic.facebook.com/groups/32275791783...,"Добрый день,подскажите",Добрый день подскажите,добрый день подскажите,подскажите,NaN
1,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша
2,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста
3,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
4,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
...,...,...,...,...,...,...,...,...,...
8010,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
8011,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
8012,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...
8013,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...


In [48]:
data=data.dropna()

In [49]:
# Reset index after drop
data=data.dropna().reset_index(drop=True)

In [50]:
data

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша
1,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста
2,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
3,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
4,2022-01-03,14,380,https://mbasic.facebook.com/groups/32275791783...,"Кто-нибудь в курсе, что за изменения в отношен...",Кто нибудь в курсе что за изменения в отношени...,кто нибудь в курсе что за изменения в отношени...,курсе отношении выдачи карты поляка,отношение выдача карта поляк
...,...,...,...,...,...,...,...,...,...
7978,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
7979,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
7980,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...
7981,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...


In [51]:
data.to_csv('C:/Users/dell/Desktop/pliki/lemm_7983.csv', encoding = 'utf-8',  index = False)

In [52]:
df = pd.read_csv(r'C:\Users\dell\Desktop\pliki\lemm_7983.csv')
df

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша
1,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста
2,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
3,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
4,2022-01-03,14,380,https://mbasic.facebook.com/groups/32275791783...,"Кто-нибудь в курсе, что за изменения в отношен...",Кто нибудь в курсе что за изменения в отношени...,кто нибудь в курсе что за изменения в отношени...,курсе отношении выдачи карты поляка,отношение выдача карта поляк
...,...,...,...,...,...,...,...,...,...
7978,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
7979,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
7980,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...
7981,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7983 entries, 0 to 7982
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           7983 non-null   object
 1   likes          7983 non-null   int64 
 2   comments       7983 non-null   int64 
 3   link to post   7983 non-null   object
 4   postfs         7983 non-null   object
 5   preproccessed  7983 non-null   object
 6   text_prep      7983 non-null   object
 7   text_sw        7983 non-null   object
 8   text_lemm      7983 non-null   object
dtypes: int64(2), object(7)
memory usage: 561.4+ KB


In [54]:
df['postfs_len'] = df['postfs'] .str.len()

In [55]:
df

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm,postfs_len
0,2022-01-02,1,70,https://mbasic.facebook.com/groups/32275791783...,Привет! Вопрос о пересечении границы Беларусь-...,Привет Вопрос о пересечении границы Беларусь П...,привет вопрос о пересечении границы беларусь п...,пересечении границы беларусь польша,пересечение граница беларусь польша,52
1,2022-01-03,13,10,https://mbasic.facebook.com/groups/32275791783...,Также делаю квесты для детей для...,Также делаю квесты для детей для,также делаю квесты для детей для,квесты детей,квеста,35
2,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...,149
3,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...,135
4,2022-01-03,14,380,https://mbasic.facebook.com/groups/32275791783...,"Кто-нибудь в курсе, что за изменения в отношен...",Кто нибудь в курсе что за изменения в отношени...,кто нибудь в курсе что за изменения в отношени...,курсе отношении выдачи карты поляка,отношение выдача карта поляк,69
...,...,...,...,...,...,...,...,...,...,...
7978,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...,182
7979,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...,137
7980,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...,242
7981,2022-07-21,0,50,https://mbasic.facebook.com/groups/32275791783...,"Всем привет Подскажите, где оформить доверенно...",Всем привет Подскажите где оформить довереннос...,всем привет подскажите где оформить довереннос...,подскажите оформить доверенность машину поеду ...,оформлять доверенность поехать польша рб знако...,127


In [56]:
df_len = df.loc[df['postfs_len'] >= 100]

In [57]:
df_len

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm,postfs_len
2,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...,149
3,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...,135
6,2022-01-03,7,70,https://mbasic.facebook.com/groups/32275791783...,Кто может выполнить комплекс строительных рабо...,Кто может выполнить комплекс строительных рабо...,кто может выполнить комплекс строительных рабо...,выполнить комплекс строительных работ нового п...,выполнять комплекс строительный помещение пека...,115
7,2022-01-03,11,220,https://mbasic.facebook.com/groups/32275791783...,"енно). Вообще, человек из жкх сказал, что обыч...",енно Вообще человек из жкх сказал что обычно е...,енно вообще человек из жкх сказал что обычно е...,енно жкх присылают какие справки польской нало...,енное жкх присылать справка польский налоговый...,339
9,2022-01-04,12,0,https://mbasic.facebook.com/groups/32275791783...,"Кто еще в раздумьях и не знает, какую полициал...",Кто еще в раздумьях и не знает какую полициаль...,кто еще в раздумьях и не знает какую полициаль...,раздумьях какую полициальную школу выбрать sta...,раздумье полициальный выбирать start центр вар...,209
...,...,...,...,...,...,...,...,...,...,...
7977,2022-07-21,24,30,https://mbasic.facebook.com/groups/32275791783...,"Здравствуйте, произошла очень неприятная ситуа...",Здравствуйте произошла очень неприятная ситуац...,здравствуйте произошла очень неприятная ситуац...,произошла неприятная ситуация автовокзале захо...,происходить неприятный ситуация автовокзал зах...,503
7978,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...,182
7979,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...,137
7980,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...,242


In [58]:
df_len = df_len.drop('postfs_len', axis=1)

In [59]:
df_len

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
2,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
3,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
6,2022-01-03,7,70,https://mbasic.facebook.com/groups/32275791783...,Кто может выполнить комплекс строительных рабо...,Кто может выполнить комплекс строительных рабо...,кто может выполнить комплекс строительных рабо...,выполнить комплекс строительных работ нового п...,выполнять комплекс строительный помещение пека...
7,2022-01-03,11,220,https://mbasic.facebook.com/groups/32275791783...,"енно). Вообще, человек из жкх сказал, что обыч...",енно Вообще человек из жкх сказал что обычно е...,енно вообще человек из жкх сказал что обычно е...,енно жкх присылают какие справки польской нало...,енное жкх присылать справка польский налоговый...
9,2022-01-04,12,0,https://mbasic.facebook.com/groups/32275791783...,"Кто еще в раздумьях и не знает, какую полициал...",Кто еще в раздумьях и не знает какую полициаль...,кто еще в раздумьях и не знает какую полициаль...,раздумьях какую полициальную школу выбрать sta...,раздумье полициальный выбирать start центр вар...
...,...,...,...,...,...,...,...,...,...
7977,2022-07-21,24,30,https://mbasic.facebook.com/groups/32275791783...,"Здравствуйте, произошла очень неприятная ситуа...",Здравствуйте произошла очень неприятная ситуац...,здравствуйте произошла очень неприятная ситуац...,произошла неприятная ситуация автовокзале захо...,происходить неприятный ситуация автовокзал зах...
7978,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
7979,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
7980,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...


In [61]:
# Reset index after drop
df_len=df_len.dropna().reset_index(drop=True)

In [62]:
df_len

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_sw,text_lemm
0,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
1,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
2,2022-01-03,7,70,https://mbasic.facebook.com/groups/32275791783...,Кто может выполнить комплекс строительных рабо...,Кто может выполнить комплекс строительных рабо...,кто может выполнить комплекс строительных рабо...,выполнить комплекс строительных работ нового п...,выполнять комплекс строительный помещение пека...
3,2022-01-03,11,220,https://mbasic.facebook.com/groups/32275791783...,"енно). Вообще, человек из жкх сказал, что обыч...",енно Вообще человек из жкх сказал что обычно е...,енно вообще человек из жкх сказал что обычно е...,енно жкх присылают какие справки польской нало...,енное жкх присылать справка польский налоговый...
4,2022-01-04,12,0,https://mbasic.facebook.com/groups/32275791783...,"Кто еще в раздумьях и не знает, какую полициал...",Кто еще в раздумьях и не знает какую полициаль...,кто еще в раздумьях и не знает какую полициаль...,раздумьях какую полициальную школу выбрать sta...,раздумье полициальный выбирать start центр вар...
...,...,...,...,...,...,...,...,...,...
5627,2022-07-21,24,30,https://mbasic.facebook.com/groups/32275791783...,"Здравствуйте, произошла очень неприятная ситуа...",Здравствуйте произошла очень неприятная ситуац...,здравствуйте произошла очень неприятная ситуац...,произошла неприятная ситуация автовокзале захо...,происходить неприятный ситуация автовокзал зах...
5628,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
5629,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
5630,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добрага дня шаноўныя нехта працуе турыстычных ...,добрага шаноўныя нехт працуй турыстычный агенц...


In [466]:
df_len.to_csv('C:/Users/dell/Desktop/pliki/lemm_5632.csv', encoding = 'utf-8',  index = False)

In [63]:
df = pd.read_csv(r'C:\Users\dell\Desktop\pliki\lemm_5632.csv')

In [64]:
df

,date,likes,comments,link to post,postfs,preproccessed,text_prep,text_stem,text_sw,text_lemm
0,2022-01-03,11,210,https://mbasic.facebook.com/groups/32275791783...,Здравствуйте! Планирую открыть счёт в банке PK...,Здравствуйте Планирую открыть счёт в банке PKO...,здравствуйте планирую открыть счёт в банке pko...,планир откр счет банк pko начислен зп подскаже...,планирую открыть счёт банке pko начисления зп ...,планировать открывать счет банк pko начисление...
1,2022-01-03,10,210,https://mbasic.facebook.com/groups/32275791783...,Всем привет. Может у кого-то из вас была такая...,Всем привет Может у кого то из вас была такая ...,всем привет может у кого то из вас была такая ...,ситуац рб числ тунеядец предостав подтвержда,ситуация рб числюсь тунеядец предоставить подт...,ситуация рб числиться тунеядец предоставлять п...
2,2022-01-03,7,70,https://mbasic.facebook.com/groups/32275791783...,Кто может выполнить комплекс строительных рабо...,Кто может выполнить комплекс строительных рабо...,кто может выполнить комплекс строительных рабо...,выполн комплекс строительн работ нов помещен п...,выполнить комплекс строительных работ нового п...,выполнять комплекс строительный помещение пека...
3,2022-01-03,11,220,https://mbasic.facebook.com/groups/32275791783...,"енно). Вообще, человек из жкх сказал, что обыч...",енно Вообще человек из жкх сказал что обычно е...,енно вообще человек из жкх сказал что обычно е...,ен жкх присыла как справк польск налогов подтв...,енно жкх присылают какие справки польской нало...,енное жкх присылать справка польский налоговый...
4,2022-01-04,12,0,https://mbasic.facebook.com/groups/32275791783...,"Кто еще в раздумьях и не знает, какую полициал...",Кто еще в раздумьях и не знает какую полициаль...,кто еще в раздумьях и не знает какую полициаль...,раздум как полициальн школ выбра start центр в...,раздумьях какую полициальную школу выбрать sta...,раздумье полициальный выбирать start центр вар...
...,...,...,...,...,...,...,...,...,...,...
5627,2022-07-21,24,30,https://mbasic.facebook.com/groups/32275791783...,"Здравствуйте, произошла очень неприятная ситуа...",Здравствуйте произошла очень неприятная ситуац...,здравствуйте произошла очень неприятная ситуац...,произошл неприятн ситуац автовокзал заходн сес...,произошла неприятная ситуация автовокзале захо...,происходить неприятный ситуация автовокзал зах...
5628,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Подскажите пожалуйста,из Тересполя в Брест,кро...",Подскажите пожалуйста из Тересполя в Брест кро...,подскажите пожалуйста из тересполя в брест кро...,подскаж тереспол брест автобус езд как маршрут...,подскажите тересполя брест автобуса ездят каки...,тересполь брест автобус ездить маршрутка приез...
5629,2022-07-21,0,40,https://mbasic.facebook.com/groups/32275791783...,"Добрый день, кто-нибудь знает, можно ли привит...",Добрый день кто нибудь знает можно ли привить ...,добрый день кто нибудь знает можно ли привить ...,прив covid желательн pfizer ’ ом родител нац виз,привить covid желательно pfizer ’ ом родителей...,прививать covid желательно pfizer ’ ом родит...
5630,2022-07-21,1,40,https://mbasic.facebook.com/groups/32275791783...,"Добрага дня, шаноўныя ! Можа тут нехта працуе ...",Добрага дня шаноўныя Можа тут нехта працуе ў т...,добрага дня шаноўныя можа тут нехта працуе ў т...,добраг дня шаноўны мож нехт працу турыстычн аг...,добрага дня шаноўныя можа нехта працуе турысты...,добрага шаноўныя можа нехт працуй турыстычный ...
